## Import the necessary libraries

In [ ]:
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip show seaborn

Name: seaborn
Version: 0.13.2
Summary: Statistical data visualization
Home-page: 
Author: 
Author-email: Michael Waskom <mwaskom@gmail.com>
License: 
Location: c:\Users\My Device\AppData\Local\Programs\Python\Python314\Lib\site-packages
Requires: matplotlib, numpy, pandas
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path



In [2]:
import seaborn as sns
print(sns.__version__)

0.13.2


In [ ]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
# Use a known matplotlib style (empty string raises an error)
plt.style.use('seaborn')        
# If running outside Jupyter, remove the next line or guard it
%matplotlib inline      

OSError: '' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

## Configuration

In [ ]:
# Configuration
TICKER = "AAPL"  # Change this to any ticker you have data for
DATA_DIR = Path("../data/raw/yfinance_data/Data/")
OUTPUT_DIR = Path("../data/processed/technical_indicators/")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

## Data Loading Module

In [ ]:
def load_stock_data(ticker, data_dir=DATA_DIR):
    """Load stock data from CSV file."""
    file_path = data_dir / f"{ticker}.csv"
    try:
        df = pd.read_csv(file_path)
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        return df.sort_index()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except Exception as e:
        print(f"Error loading {ticker}: {str(e)}")
        return None

NameError: name 'DATA_DIR' is not defined

## Technical indicators

In [ ]:
# 4.1 Moving Averages
def add_moving_averages(df, windows=[20, 50]):
    """Add Simple Moving Averages to dataframe."""
    for window in windows:
        df[f'SMA_{window}'] = df['Close'].rolling(window=window).mean()
    return df

# 4.2 RSI
def add_rsi(df, periods=14):
    """Add Relative Strength Index to dataframe."""
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

# 4.3 MACD
def add_macd(df, fast=12, slow=26, signal=9):
    """Add MACD and Signal Line to dataframe."""
    df['EMA_12'] = df['Close'].ewm(span=fast, adjust=False).mean()
    df['EMA_26'] = df['Close'].ewm(span=slow, adjust=False).mean()
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['Signal_Line'] = df['MACD'].ewm(span=signal, adjust=False).mean()
    return df

# 4.4 Bollinger Bands
def add_bollinger_bands(df, window=20, num_std=2):
    """Add Bollinger Bands to dataframe."""
    df['BB_Middle'] = df['Close'].rolling(window=window).mean()
    df['BB_Upper'] = df['BB_Middle'] + (df['Close'].rolling(window=window).std() * num_std)
    df['BB_Lower'] = df['BB_Middle'] - (df['Close'].rolling(window=window).std() * num_std)
    return df

## Data Processing

In [ ]:
# Load the data
df = load_stock_data(TICKER)

if df is not None:
    # Add all indicators
    df = add_moving_averages(df)
    df = add_rsi(df)
    df = add_macd(df)
    df = add_bollinger_bands(df)
    
    # Display first few rows with indicators
    display(df[['Close', 'SMA_20', 'SMA_50', 'RSI', 'MACD', 'Signal_Line']].head())
    
    # Save the results
    output_file = OUTPUT_DIR / f"{TICKER}_indicators.csv"
    df.to_csv(output_file)
    print(f"Saved indicators to {output_file}")
else:
    print("Failed to load data. Please check the ticker symbol and data directory.")